In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math 
import os

In [34]:
def margin_req(strike,price,premium,n_options = 100, type = "put"):
    if type == "put":
        # let's find the maximum between 3 choices:
        max_choice = np.max([0.3 * price - np.max([price - strike,0]),
               0.1 * strike,
               2.50 * n_options])
        return (max_choice + premium) * 100
    elif type == "call":
         # let's find the maximum between 3 choices:
        max_choice = np.max([0.3 * price - np.max([-(price - strike),0]),
               0.1 * strike,
               2.50 * n_options])
        return (max_choice + premium) * 100


In [35]:
margin_req(90,100,1,n_options=1,type="put")

2100.0

In [36]:
# checking the margin requirement for a put option that evolves with the price of the underlying
price_evol = [100,95,90,85,40,50]
premium_evol = [1,2,4,7,50.5,40]
for i in range(0,len(price_evol)):
    print(margin_req(90,price_evol[i],premium_evol[i],n_options=1,type="put"))

2100.0
2550.0
3100.0
3250.0
6250.0
5500.0


In [37]:
# checking the margin requirement for a call option that evolves with the price of the underlying
price_evol = [40,45,50,55,90,100]
premium_evol = [1,2,4,7,40.5,50]
for i in range(0,len(price_evol)):
    print(margin_req(50,price_evol[i],premium_evol[i],n_options=1,type="call"))

600.0
1050.0
1900.0
2350.0
6750.0
8000.0


In [40]:
pwd

'c:\\Users\\HP\\source\\repos\\optionstrade'

In [45]:
position_jan_11_25 = pd.read_csv("data/positions_11_01_2025.csv", engine='python', encoding = "ISO-8859-1")

In [46]:
position_jan_11_25.head()

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75
1,SPDR S&P 500 ETF TRUST,NaN,USD,USD,USD,USD,USD,NaN
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73
3,ISHARES BITCOIN TRUST ETF,NaN,USD,USD,USD,USD,USD,NaN
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25


In [47]:
position_jan_11_25["Instrument\xa0"]

0            SPY Jan17'25 585 Put 
1           SPDR S&P 500 ETF TRUST
2            IBIT Mar21'25 51 Put 
3        ISHARES BITCOIN TRUST ETF
4            SPY Mar21'25 545 Put 
5           SPDR S&P 500 ETF TRUST
6          CAR.UN Feb21'25 48 Put 
7     CAN APARTMENT PROP REAL ESTA
8            IBIT Jan17'25 55 Put 
9        ISHARES BITCOIN TRUST ETF
10           IWM Jan17'25 219 Put 
11        ISHARES RUSSELL 2000 ETF
12          MSFT Jan17'25 420 Put 
13                  MICROSOFT CORP
14          NVDA Jan17'25 136 Put 
15                     NVIDIA CORP
16          RCAT Apr17'25 15 Call 
17            RED CAT HOLDINGS INC
18           CRNT Mar21'25 6 Call 
19            CERAGON NETWORKS LTD
20          IONQ Apr17'25 55 Call 
21                        IONQ INC
22         NVDA Apr17'25 150 Call 
23                     NVIDIA CORP
24         ANET Mar21'25 120 Call 
25             ARISTA NETWORKS INC
26                           IBIT 
27       ISHARES BITCOIN TRUST ETF
28         RDDT Mar2

In [62]:
position_jan_11_25["type"] = position_jan_11_25["Instrument\xa0"].str.split().str[-1]

In [82]:
for y in range(0,len(position_jan_11_25["type"])):
    #for i in position_jan_11_25["type"].unique():
    if position_jan_11_25["type"][y] == "Put":
        position_jan_11_25["type0"][y] = "option" 
    elif position_jan_11_25["type"][y] == "Call":
        position_jan_11_25["type0"][y] = "option"
    else: 
        position_jan_11_25["type0"][y] = "Not option"

In [83]:
position_jan_11_25.head()

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75,Put,option
1,SPDR S&P 500 ETF TRUST,NaN,USD,USD,USD,USD,USD,NaN,TRUST,Not option
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73,Put,option
3,ISHARES BITCOIN TRUST ETF,NaN,USD,USD,USD,USD,USD,NaN,ETF,Not option
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25,Put,option


In [84]:
option_position_jan_11_25 = position_jan_11_25[position_jan_11_25["type0"] == "option"]

In [86]:
option_position_jan_11_25.head(10)

,Instrument,Position,Cost Basis,Unrealized P&L,Market Value,Avg Price,Daily P&L,Last,type,type0
0,SPY Jan17'25 585 Put,-27,"-8,513.10","-12,030.00","-21,455.60",3.15,"-11,150.00",7.75,Put,option
2,IBIT Mar21'25 51 Put,-50,"-17,722",-931,"-18,866.18",3.54,"1,850.00",3.73,Put,option
4,SPY Mar21'25 545 Put,-20,"-8,565.64","-3,970.00","-12,691.09",4.28,"-3,560.00",6.25,Put,option
6,CAR.UN Feb21'25 48 Put,-20,"-5,375.00","-7,260.00","-12,636.27",2.69,"-1,540.00",C5.55,Put,option
8,IBIT Jan17'25 55 Put,-60,"-9,170.94","-3,190.00","-12,360.81",1.53,"3,130.00",2.01,Put,option
10,IWM Jan17'25 219 Put,-3,-643.45,-611,"-1,357.62",2.14,-651,4.14,Put,option
12,MSFT Jan17'25 420 Put,-2,-659.54,-420,"-1,094.96",3.3,-414,5.45,Put,option
14,NVDA Jan17'25 136 Put,-3,-608.62,-343,"-1,010.76",2.03,-367,3.2,Put,option
16,RCAT Apr17'25 15 Call,20,"8,995.57","-5,380.00","3,617.67",4.5,"-1,590.00",1.8,Call,option
18,CRNT Mar21'25 6 Call,200,"12,157","-6,480.00","5,706.23",0.61,-822,0.3,Call,option


In [81]:
for y in range(0,len(position_jan_11_25["type"])):
    if position_jan_11_25["type"][y] == "Put":
        print("put")
    else:
        print("nope")

put
nope
put
nope
put
nope
put
nope
put
nope
put
nope
put
nope
put
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
nope
